Арсений Коровко

Итоговое решение можно разбить на следующие этапы:

1) Первичная обработка данных: загружаем файлы, заменяем пропуски в таргете на нули, лемматизируем тексты, удаляем из текстов стоп-слова и пунктуацию

2) Создаем класс модели со встроенным трансформером. Трансформер производит tf-idf трансформацию текстов, а затем применяет к полученному PCA; модель - One Versus Rest версия любого классификатора, с поддержкой predict и predict_proba по определенному порогу. Класс обучает трансформер, а затем модель на тренировочной выборке, после чего применяет к любой полученной

3) Выбираем лучшие параметры для класса модели с трансформером с помощью кросс-валидации(3 фолда)

Лучший результат - accuracy в диапазоне от 0.61 до 0.65(и на кросс-валидации, и на всей выборке) Подобные значения достигаются при количестве компонент PCA около 65, классификатором SVC с С около 7.

Вещи, которые были опробованы, но не привели к лучшим результатам: использовать альтернативные ядра для SVC, использовать логистическую регрессию, не использовать PCA, добавить нормированную длину предложения как признак, использовать предобученные эмбеддинги слов с deeppavlov.ai для формирования признаков(min/max для каждой координаты по векторам эмбеддингов слов в предложении).

In [174]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('punkt')
stoplist = set(stopwords.words('russian'))
punctuation = re.compile('[0-9\\\\!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~]')
import re
import string
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from nltk.corpus import stopwords
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import cross_validate
from pymystem3 import Mystem

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Арсений\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Арсений\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Первичная обработка:

In [ ]:
def preprocess(corpus):
    for i in range(len(corpus)):
        text = corpus[i]
        mystem = Mystem()
        text = mystem.lemmatize(text)
        text = ''.join([w for w in text if not w in stoplist])
        text = text.replace('\n', ' ').lower()
        text = punctuation.sub(' ', text)
        text = ' '.join([w for w in text.split()])
        corpus[i] = text
    return corpus

In [ ]:
df = pd.read_csv("data_train_heart.csv", encoding="utf8")
df_add = pd.read_csv("add_data.csv", encoding="utf8")
data = pd.concat([df, df_add], axis=0)
data = data.fillna(0)
X = preprocess(list(data["text"]))
y = data.drop(["text", "id"], axis=1)
X_data = pd.DataFrame(X, columns=['text'])

Блок снизу использует датасет эмбеддингов с http://docs.deeppavlov.ai/en/master/features/pretrained_vectors.html

Можно не запускать его, если нет необходимости - он не используется в финальной модели

In [642]:
with open('ft_native_300_ru_wiki_lenta_lemmatize.vec', encoding='utf-8') as f:
    embed_data = f.readlines()
embed_data = {elt.split()[0]: np.array(elt.split()[1:]).astype(float) for elt in embed_data}

Класс модели с трансформером и функция для кросс-валидации:

In [678]:
class DataTransformer:
    def __init__(self, n_pca, ignore_pca=False, add_text_lens=False, add_word_embeds_minmax=False):
        self.vectorizer = TfidfVectorizer()
        if not ignore_pca:
            self.pca = TruncatedSVD(n_components=n_pca)
        self.ignore_pca = ignore_pca
        self.add_text_lens = add_text_lens
        self.add_word_embeds_minmax = add_word_embeds_minmax
    
    def get_minmax_vecs(self, X):
            feat_vecs = []
            for sent in list(X['text']):
                word_vecs = []
                for word in sent:
                    try:
                        word_vecs.append(embed_data[word])
                    except:
                        pass
                word_vecs = np.array(word_vecs)
                minmax_feats = np.concatenate([np.max(word_vecs, axis=0), np.min(word_vecs, axis=0)])
                feat_vecs.append(minmax_feats)
            return np.array(feat_vecs)
    
    def fit(self, X):
        X_new = self.vectorizer.fit_transform(list(X['text']))
        if self.add_text_lens:
            text_lens = np.array(list(map(lambda x: len(x), list(X['text']))))
            self.max_text_len = text_lens.max()
            X_new_2 = np.zeros((X_new.shape[0], X_new.shape[1]+1))
            X_new_2[:,:-1] = X_new.todense()
            X_new_2[:, -1] = text_lens/self.max_text_len
            X_new = X_new_2
        if self.add_word_embeds_minmax:
            feat_vecs = self.get_minmax_vecs(X)
            X_new_2 = np.zeros((X_new.shape[0], X_new.shape[1]+feat_vecs.shape[1]))
            X_new_2[:,:X_new.shape[1]] = X_new.todense() # will not work with self.add_text_lens = True
            X_new_2[:, X_new.shape[1]:] = feat_vecs
            X_new = X_new_2
        if not self.ignore_pca:
            self.pca.fit(X_new)
    
    def transform(self, X):
        X_new = self.vectorizer.transform(list(X['text']))
        if self.add_text_lens:
            text_lens = np.array(list(map(lambda x: len(x), list(X['text']))))
            X_new_2 = np.zeros((X_new.shape[0], X_new.shape[1]+1))
            X_new_2[:,:-1] = X_new.todense()
            X_new_2[:, -1] = text_lens/self.max_text_len
            X_new = X_new_2
        if self.add_word_embeds_minmax:
            feat_vecs = self.get_minmax_vecs(X)
            X_new_2 = np.zeros((X_new.shape[0], X_new.shape[1]+feat_vecs.shape[1]))
            X_new_2[:,:X_new.shape[1]] = X_new.todense() # will not work with self.add_text_lens = True
            X_new_2[:, X_new.shape[1]:] = feat_vecs
            X_new = X_new_2
        if self.ignore_pca:
            return X_new
        return self.pca.transform(X_new)
    
    def fit_transform(self, X):
        self.fit(X)
        return self.transform(X)
    
class ModelWithTransform:
    def __init__(self, model, n_pca, ignore_pca=False, add_text_lens=False,
                 predict_proba=False, proba_threshold=0.35, add_word_embeds_minmax=False):
        self.transformer = DataTransformer(n_pca, ignore_pca, add_text_lens, add_word_embeds_minmax)
        self.model = model
        self.predict_proba = predict_proba
        self.proba_threshold = proba_threshold
    
    def fit(self, X, y):
        X = self.transformer.fit_transform(X)
        self.model.fit(X, y)
    
    def predict(self, X):
        X = self.transformer.transform(X)
        if not self.predict_proba:
            return self.model.predict(X)
        else:
            return (self.model.predict_proba(X)>self.proba_threshold).astype(int)

def cross_val_acc(model, X, y, cv=3):
    n_els = y.shape[0]
    step_size = int(n_els/cv)
    inds = np.arange(n_els)
    np.random.shuffle(inds)
    accs = []
    for i in range(cv):
        inds_tr = inds[step_size*i:step_size*(i+1)*(i!=n_els-1)+n_els*(i==n_els-1)]
        model.fit(X.iloc[~inds_tr], y.iloc[~inds_tr])
        accs.append(accuracy_score(y.iloc[inds_tr], model.predict(X.iloc[inds_tr])))
    return np.array(accs)

Тестирование моделей:

In [777]:
"""model_base = OneVsRestClassifier(LogisticRegression(max_iter=3000))
model_0 = ModelWithTransform(model_base, n_pca=60, ignore_pca=False, predict_proba=True,
                             proba_threshold=0.3, add_word_embeds_minmax=True)"""
model_base = OneVsRestClassifier(SVC(C=7))
model_0 = ModelWithTransform(model_base, n_pca=65)
cv_results = cross_val_acc(model_0, X_data, y, cv=3)
print(cv_results)
print('Accuracy: {:.2f}'.format(cv_results.mean()))

[0.62459547 0.6407767  0.64401294]
Accuracy: 0.64


In [778]:
y_pred = model_0.predict(X_data)
print('Accuracy: {:.2f}'.format(accuracy_score(y, y_pred)))

Accuracy: 0.62


Выводим описания объектов, на которых модель допускает ошибки:

In [612]:
errs = (y!=y_pred).sum(axis=1)
errs = errs[errs!=0]
list(data.loc[errs, 'text'])

['Текст: PER обратился по вопросу онкобольной жены (NUMBER лет). Диагноз ра кишечника был поставлен в NUMBERг, проведена операция. Лечение больше не оказывается, прописана хт в таблетках. PER не уверен в компетентности врачей своего города, необходима консультация специалиста. \nОжидаем дополнение с сайта.\n\nИнформация о фонде получена из интернета по запросу ORG.\n',
 'Текст обращения: У близкого человека PER онкология NUMBER ст, есть проблема, что появляется оттек, хотели спросить куда обращаться, чтобы его не было. PER рассказала, что NUMBER дней назад обращались ко врачу в больницу, которая находится далеко от них, там им дали направление на госпитализацию, но он отказался. Я порекомендовала PER обратиться к местному терапевту, чтобы он дал направление к онкологу для наблюдения онкопациента. Я поняла из нашего разговора, что онкопациент отказывается от лечения, так как он даже не наблюдается у врачей, только иногда. Хотела с PER поговорить на тему отказа лечения больного, но она н